In [25]:
from uuid import uuid4
from modules.config import *
from modules.filemanager import read_json_file
from modules.sql import *

foods_data = read_json_file(FOOD_FILE_JSON)["foods"]
nutrients_data = db_query("select * from nutrienttype")
ingredientCategories = db_query("select * from ingredientcategory")

messure_dl_id = db_query_single("select id from messure where shortname = 'dl'")[0]
messure_skive_id = db_query_single("select id from messure where shortname = 'skive'")[0]
messure_stk_id = db_query_single("select id from messure where shortname = 'stk'")[0]
messure_g_id = db_query_single("select id from messure where shortname = 'g'")[0]

nutrient_kj_id = db_query_single("select id from nutrienttype where name = 'energi'")[0]
nutrient_kcal_id = db_query_single("select id from nutrienttype where name = 'kalorier'")[0]

In [26]:
def query_food(datalist: list[dict], query_string):
    indexed_result = []
    for i,v in enumerate(datalist):
        if query_string.lower() in v["foodName"].lower():
            indexed_result.append((i,v["foodName"]))
    for v in indexed_result:
        print(v)

def resolve_messure(portions: list[dict]):
    for p in portions:
        if p["portionName"] == "desiliter": return messure_dl_id, p["quantity"]
        if p["portionName"] == "stk": return messure_stk_id, p["quantity"]
        if p["portionName"] == "skive": return messure_skive_id, p["quantity"]
        return messure_g_id, 1

In [27]:
for i,v in enumerate(ingredientCategories):
    print(f"{i} - {v[1]}")

0 - frukt
1 - grønnsak
2 - gryn
3 - meieri
4 - melk


In [28]:
# select categories for food item
select_categories = [0,1]

In [37]:
# Enter food to query
name_query = "eple"

exist_in_db = db_query_single(f"select * from ingredient where name = '{name_query}'") != None

query_food(foods_data, name_query)


(74, 'Barnemat, flerkornsgrøt med eple og pære, fra 12 mnd, pulver, Småfolk')
(371, 'Eple, Granny Smith, Golden Delicious, rå')
(372, 'Eple, Ingrid Marie, rå')
(373, 'Eple, Pink lady, rå')
(374, 'Eple, Rød aroma, rå')
(375, 'Eple, hermetisk')
(376, 'Eple, importert, rå')
(377, 'Eple, norsk, uspesifisert, rå')
(378, 'Eple, tørket')
(379, 'Eple, uspesifisert, rå')
(380, 'Eplecidereddik')
(381, 'Eplejuice, most')
(382, 'Eplekake')
(383, 'Eplemos')
(384, 'Eplemos, lett')
(385, 'Eplemos, uten tilsatt sukker')
(386, 'Eplenektar')
(387, 'Eplepai')
(479, 'Fyrstekake, med epler')
(511, 'Granateple, rå')
(512, 'Granateplekjerner, fryst')
(724, 'Juice, med eple, gulrot, ingefær og sitron, kjøpt i kafé/bakeri')
(1301, 'Paradisepler, rå')
(1785, 'Smoothie, blåbær og eple')
(1794, 'Smuldrepai, med eple')
(1937, 'Syrnet melk, rødbete/eple/lime, Biola')


In [38]:
# select index for food and selected entity name
# source uri will stop any dupicate entries
index = 377
entity_name = ""


if not entity_name: entity_name = name_query

# get data to process for selected index
data = foods_data[index]


data_source = data["uri"]
data_portions = data["portions"]
data_energy = data["energy"]
data_calories = data["calories"]
data_nutrients = data["constituents"]
data_nutrients = [x for x in data_nutrients if x["sourceId"] != "10"]

messure_id, ratio = resolve_messure(data_portions)

In [39]:
# create ingredient entity
ingredient_entity = {
    "id": str(uuid4()),
    "name": entity_name.lower(),
    "volumeWeightRatio": ratio,
    "messureId": messure_id,
}

ingredient_entity

{'id': 'afffebb1-388d-414d-aabc-95b1ce01ec50',
 'name': 'eple',
 'volumeWeightRatio': 1,
 'messureId': 'ddcae6e7-26a9-11f0-b7b0-d85ed33237df'}

In [40]:
# create source entity
sourcetypeId = db_query_single("select id from sourcetype where type = 'href'")[0]

source_entity = {
    "id": str(uuid4()),
    "text": data_source,
    "typeId": sourcetypeId,
    "ingredientId": ingredient_entity["id"]
}

source_entity

{'id': 'b243bfbc-7002-4490-b10e-e3a4a8daf68a',
 'text': 'https://www.matvaretabellen.no/eple-norsk-uspesifisert-ra/',
 'typeId': 'ddd16f90-26a9-11f0-b7b0-d85ed33237df',
 'ingredientId': 'afffebb1-388d-414d-aabc-95b1ce01ec50'}

In [41]:
# create entities for ingredient categories selected
ic_entities = []
for index in select_categories:
    ic_entities.append({
        "id": str(uuid4()),
        "IngredientId": ingredient_entity["id"],
        "ingredientCategoryId": ingredientCategories[index][0]
    })
    
ic_entities

[{'id': '975625e7-658f-4670-a048-9855d779ccad',
  'IngredientId': 'afffebb1-388d-414d-aabc-95b1ce01ec50',
  'ingredientCategoryId': 'dde0b8dc-26a9-11f0-b7b0-d85ed33237df'},
 {'id': '7a603755-253e-44e5-bc7f-8253f3d52759',
  'IngredientId': 'afffebb1-388d-414d-aabc-95b1ce01ec50',
  'ingredientCategoryId': 'dde0b7ef-26a9-11f0-b7b0-d85ed33237df'}]

In [42]:
# add energy and calories to entities
nutrients_entities = []

nutrients_entities.append({
    "id": str(uuid4()),
    "ingredientId": ingredient_entity["id"],
    "nutrientTypeId": nutrient_kj_id,
    "value": data_energy["quantity"]
})

nutrients_entities.append({
    "id": str(uuid4()),
    "ingredientId": ingredient_entity["id"],
    "nutrientTypeId": nutrient_kcal_id,
    "value": data_calories["quantity"]
})

# and remaining nutrients
for n in data_nutrients:
    if n["quantity"] == 0:
        continue
    nd_id = [x for x in nutrients_data if x[2] == n["nutrientId"]][0][0]
    result = {
        "id": str(uuid4()),
        "ingredientId": ingredient_entity["id"],
        "nutrientTypeId": nd_id,
        "value": n["quantity"]
    }
    nutrients_entities.append(result)

print(len(nutrients_entities))

26


In [43]:
# ensure to run db insert here!!!
exist = db_query_single(f"select * from ingredientnutrientsource where text = '{source_entity['text']}'") != None

In [44]:
# insert table data
if not exist:
    db_insert("ingredient", list(ingredient_entity.keys()), [list(ingredient_entity.values())])
    db_insert("ingredientnutrientsource", list(source_entity.keys()), [list(source_entity.values())])

    ic_keys = list(ic_entities[0].keys())
    ic_values = [list(x.values()) for x in ic_entities]
    db_insert("ingredient_ingredientcategory", ic_keys, ic_values)

    nutr_keys= list(nutrients_entities[0].keys())
    nutr_values = [list(x.values()) for x in nutrients_entities]
    db_insert("nutrientingredient", nutr_keys, nutr_values)
    
    print("! Data inserted !")
else:
    print("Item existed in database")

! Data inserted !
